<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Speech_and_Text_Fusion_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Speech and Text Fusion Models:


Combining Audio, Speech, and Textual Data
Case: Sentiment Analysis on Customer Support Calls
We'll build a Speech-Text Fusion Model to perform sentiment analysis on customer support calls. We’ll use Wav2Vec2 to extract features from audio and BERT to handle text data. We’ll fuse these features using a simple concatenation layer and fine-tune the model for sentiment analysis.

In [1]:
!pip install transformers torch torchaudio datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model, BertModel, BertTokenizer
from datasets import load_dataset

# Load pre-trained Wav2Vec2 and BERT models
wav_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Load a sample audio-text sentiment dataset (e.g., emotion from Hugging Face)
dataset = load_dataset("emotion", split="train[:5%]")  # For demo purposes


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
def preprocess_data(examples):
    # Process audio with Wav2Vec2
    audio_input = wav_processor(examples["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_values
    audio_features = wav_model(audio_input).last_hidden_state.mean(dim=1)

    # Process text with BERT
    text_input = bert_tokenizer(examples["text"], padding=True, truncation=True, return_tensors="pt")
    text_features = bert_model(**text_input).last_hidden_state.mean(dim=1)

    # Concatenate features
    fused_features = torch.cat((audio_features, text_features), dim=1)
    return {"features": fused_features, "labels": examples["label"]}

dataset = dataset.map(preprocess_data)
dataset.set_format(type="torch", columns=["features", "labels"])


Parameter 'function'=<function preprocess_data at 0x7837115776d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

KeyError: 'audio'

In [4]:
import torch.nn as nn
import torch.optim as optim

class FusionClassifier(nn.Module):
    def __init__(self):
        super(FusionClassifier, self).__init__()
        self.fc1 = nn.Linear(768 + 768, 256)  # Assuming 768-dim from both Wav2Vec2 and BERT
        self.fc2 = nn.Linear(256, 3)  # 3 classes for sentiment (e.g., positive, neutral, negative)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, criterion, and optimizer
fusion_model = FusionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fusion_model.parameters(), lr=1e-4)


In [5]:
for epoch in range(3):  # 3 epochs for demo
    for batch in dataset:
        optimizer.zero_grad()
        outputs = fusion_model(batch["features"])
        loss = criterion(outputs, batch["labels"])
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


KeyError: 'features'